In [1]:
import openai

In [2]:
openai.__version__

'1.70.0'

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel,Agent,Runner,set_default_openai_client
from agents.model_settings import ModelSettings
from contextlib import AsyncExitStack

[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 99] Cannot assign requested address

In [4]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)

from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel,Agent,Runner,set_default_openai_client
from agents.model_settings import ModelSettings
from agents import function_tool
import requests,json
import pymysql

from agents import (
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

from IPython.display import display, Code, Markdown, Image

In [5]:
import os
import openai
import glob
import shutil

import numpy as np
import pandas as pd
import pymysql

import json
import io
import inspect
import requests
import re
import random
import string
import base64

from bs4 import BeautifulSoup
import dateutil.parser as parser
import tiktoken
from lxml import etree

import sys
from dotenv import load_dotenv
from openai import OpenAI

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Code, Markdown, Image
from IPython import get_ipython

- DeepSeek API调用流程回顾

In [6]:
# 读取模型API-KEY
API_KEY = os.getenv("API_KEY")

# 实例化客户端
client = OpenAI(api_key=API_KEY, 
                base_url="https://api.deepseek.com")

# 调用 deepseekv3 模型
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": "你好，好久不见!请介绍下你自己。"}
    ]
)

# 输出生成的响应内容
print(response.choices[0].message.content)

你好呀！确实好久不见～很高兴再次和你聊天！😊  

我是 **DeepSeek Chat**，由深度求索公司打造的智能AI助手。我的最新版本是 **DeepSeek-V3**，知识更新至 **2024年7月**，拥有 **128K 上下文记忆**，可以处理超长文本，还能阅读并分析 **PDF、Word、Excel、PPT** 等文件。  

✨ **我的特点**：  
- **免费使用**：目前不收费，放心提问！  
- **超强理解力**：能解答技术、学习、生活、娱乐等各种问题。  
- **文件阅读**：上传文档，我可以帮你总结、提取信息或分析内容。  
- **持续进步**：团队在持续优化我，让我更聪明、更贴心！  

你可以问我任何问题，比如最新科技动态、学习建议、写作灵感，或者单纯聊聊天～ 😃 最近有什么我可以帮你的吗？


- Agents SDK基本调用流程回顾

In [7]:
external_client = AsyncOpenAI(
    base_url = "https://api.deepseek.com",
    api_key=API_KEY, 
)

set_default_openai_client(external_client)

deepseek_model = OpenAIChatCompletionsModel(
    model="deepseek-chat",
    openai_client=external_client)

In [8]:
agent = Agent(name="Assistant", 
              instructions="你是一名助人为乐的助手。",
              model=deepseek_model)

result = await Runner.run(agent, "请写一首关于编程中递归的俳句。") 

print(result.final_output)

函数自调用，  
层层深入栈深处，  
基线终有归。


## 一、MCP入门介绍与接入Agents SDK基本流程

&emsp;&emsp;在3月27号，Agents SDK正式官宣支持MCP使用，这也使得Agents SDK的实际应用场景得到待拓展：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250401201612956.png" alt="image-20250401201612956" style="zoom:33%;" />

现在，我们仅需在创建Agent的时候，将MCP服务器视作为一项工具，即可顺利调用MCP服务器进行Agent开发。而实际在借助Agents SDK调用MCP的流程也非常简单，我们`只需将MCP视作tools`，即可进行调用。换而言之，就是如果使用Agents SDK作为Agent开发框架，则可以零门槛快速接入MCP海量服务器生态。

### 1.MCP技术回顾

&emsp;&emsp;MCP，全称是Model Context Protocol，模型上下文协议，由Claude母公司Anthropic于去年11月正式提出。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318201338022.png" alt="image-20250318201338022" style="zoom:33%;" />

> - Anthropic MCP发布通告：https://www.anthropic.com/news/model-context-protocol
> - MCP GitHub主页：https://github.com/modelcontextprotocol

MCP的核心作用，是统一了Agent开发过程中，大模型调用外部工具的技术实现流程，从而大幅提高Agent开发效率。在MCP诞生之前，不同的外部工具各有不同的调用方法，要连接这些外部工具开发Agent，就必须“每一把锁单独配一把钥匙”，开发工作非常繁琐：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403170211085.png" alt="image-20250403170211085" style="zoom:33%;" />

而MCP的诞生，则统一了这些外部工具的调用流程，使得无论什么样的工具，都可以借助MCP技术按照统一的一个流程快速接入到大模型中，从而大幅加快Agent开发效率。这就好比现在很多设备都可以使用type-c和电脑连接类似。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403170238895.png" alt="image-20250403170238895" style="zoom:33%;" />

从技术实现角度来看，我们可以将MCP看成是Function calling的一种封装，通过server-client架构和一整套开发工具，来规范化Function calling开发流程。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318202116026.png" alt="image-20250318202116026" style="zoom:33%;" />

换而言之，Agents SDK实现MCP技术流程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403171429137.png" alt="image-20250403171429137" style="zoom:50%;" />

除此之外，实际上，MCP提供了三个方面的技术支持，其一是抽象的协议，也就是技术理论层面的设计；其二是一整套MCP的开发工具，也就是一些库，方便大家进行MCP工具（MCP Server）的开发并构建智能体；其三，MCP还提供了一整套非常丰富的开发生态，由于MCP是一套标准协议，任何MCP工具（MCP Server）都可以无缝接入任何MCP智能体中，也就是你开发的工具我也能用，我开发的工具你也能直接用。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403170307263.png" alt="image-20250403170307263" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403171954498.png" alt="image-20250403171954498" style="zoom:33%;" />

### 3.MCP基础实践流程

&emsp;&emsp;接下来，我们先尝试手动实现一遍MCP实践流程，然后再考虑将已经部署好的server带入Agents SDK中，作为tools进行调用。

&emsp;&emsp;一个极简的天气查询MCP调用流程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318172155677.png" alt="image-20250318172155677" style="zoom:50%;" />

- 借助uv创建MCP运行环境

**方法 1：使用 `pip` 安装（适用于已安装 `pip` 的系统）**

```bash
pip install uv
```

**方法 2：使用 `curl` 直接安装**

如果你的系统没有 `pip`，可以直接运行：

```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

这会自动下载 `uv` 并安装到 `/usr/local/bin`。

- 创建 MCP 客户端项目

```bash
# 创建项目目录
uv init mcp-client
cd mcp-client
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202503171503701.png" alt="image-20250317150300621" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202503171503750.png" alt="image-20250317150324701" style="zoom:50%;" />

- 创建MCP客户端虚拟环境

```bash
# 创建虚拟环境
uv venv

# 激活虚拟环境
source .venv/bin/activate
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202503171509604.png" alt="image-20250317150947534" style="zoom:50%;" />

然后即可通过add方法在虚拟环境中安装相关的库。

```bash
# 安装 MCP SDK
uv add mcp openai python-dotenv httpx
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202503171510547.png" alt="image-20250317151039345" style="zoom:50%;" />

- 编写用于天气查询的server服务器代码：

这里我们需要在服务器上创建一个server.py，并写入如下代码：

```python
import json
import httpx
from typing import Any
from mcp.server.fastmcp import FastMCP

# 初始化 MCP 服务器
mcp = FastMCP("WeatherServer")

# OpenWeather API 配置
OPENWEATHER_API_BASE = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "YOUR_API_KEY"  # 请替换为你自己的 OpenWeather API Key
USER_AGENT = "weather-app/1.0"

async def fetch_weather(city: str) -> dict[str, Any] | None:
    """
    从 OpenWeather API 获取天气信息。
    :param city: 城市名称（需使用英文，如 Beijing）
    :return: 天气数据字典；若出错返回包含 error 信息的字典
    """
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric",
        "lang": "zh_cn"
    }
    headers = {"User-Agent": USER_AGENT}

    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(OPENWEATHER_API_BASE, params=params, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()  # 返回字典类型
        except httpx.HTTPStatusError as e:
            return {"error": f"HTTP 错误: {e.response.status_code}"}
        except Exception as e:
            return {"error": f"请求失败: {str(e)}"}

def format_weather(data: dict[str, Any] | str) -> str:
    """
    将天气数据格式化为易读文本。
    :param data: 天气数据（可以是字典或 JSON 字符串）
    :return: 格式化后的天气信息字符串
    """
    # 如果传入的是字符串，则先转换为字典
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except Exception as e:
            return f"无法解析天气数据: {e}"

    # 如果数据中包含错误信息，直接返回错误提示
    if "error" in data:
        return f"⚠️ {data['error']}"

    # 提取数据时做容错处理
    city = data.get("name", "未知")
    country = data.get("sys", {}).get("country", "未知")
    temp = data.get("main", {}).get("temp", "N/A")
    humidity = data.get("main", {}).get("humidity", "N/A")
    wind_speed = data.get("wind", {}).get("speed", "N/A")
    # weather 可能为空列表，因此用 [0] 前先提供默认字典
    weather_list = data.get("weather", [{}])
    description = weather_list[0].get("description", "未知")

    return (
        f"🌍 {city}, {country}\n"
        f"🌡 温度: {temp}°C\n"
        f"💧 湿度: {humidity}%\n"
        f"🌬 风速: {wind_speed} m/s\n"
        f"🌤 天气: {description}\n"
    )

@mcp.tool()
async def query_weather(city: str) -> str:
    """
    输入指定城市的英文名称，返回今日天气查询结果。
    :param city: 城市名称（需使用英文）
    :return: 格式化后的天气信息
    """
    data = await fetch_weather(city)
    return format_weather(data)

if __name__ == "__main__":
    # 以标准 I/O 方式运行 MCP 服务器
    mcp.run(transport='stdio')
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318174907749.png" alt="image-20250318174907749" style="zoom: 33%;" />

- 天气查询客户端client创建流程

然后创建一个可以和server进行通信的客户端，需要注意的是，该客户端需要包含大模型调用的基础信息。我们需要编写一个client.py脚本，聂荣如下：

```python
import asyncio
import os
import json
from typing import Optional
from contextlib import AsyncExitStack

from openai import OpenAI  
from dotenv import load_dotenv

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# 加载 .env 文件，确保 API Key 受到保护
load_dotenv()

class MCPClient:
    def __init__(self):
        """初始化 MCP 客户端"""
        self.exit_stack = AsyncExitStack()
        self.openai_api_key = os.getenv("OPENAI_API_KEY")  # 读取 OpenAI API Key
        self.base_url = os.getenv("BASE_URL")  # 读取 BASE YRL
        self.model = os.getenv("MODEL")  # 读取 model
        if not self.openai_api_key:
            raise ValueError("❌ 未找到 OpenAI API Key，请在 .env 文件中设置 OPENAI_API_KEY")
        self.client = OpenAI(api_key=self.openai_api_key, base_url=self.base_url) # 创建OpenAI client
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()        

    async def connect_to_server(self, server_script_path: str):
        """连接到 MCP 服务器并列出可用工具"""
        is_python = server_script_path.endswith('.py')
        is_js = server_script_path.endswith('.js')
        if not (is_python or is_js):
            raise ValueError("服务器脚本必须是 .py 或 .js 文件")

        command = "python" if is_python else "node"
        server_params = StdioServerParameters(
            command=command,
            args=[server_script_path],
            env=None
        )

        # 启动 MCP 服务器并建立通信
        stdio_transport = await self.exit_stack.enter_async_context(stdio_client(server_params))
        self.stdio, self.write = stdio_transport
        self.session = await self.exit_stack.enter_async_context(ClientSession(self.stdio, self.write))

        await self.session.initialize()

        # 列出 MCP 服务器上的工具
        response = await self.session.list_tools()
        tools = response.tools
        print("\n已连接到服务器，支持以下工具:", [tool.name for tool in tools])     
        
    async def process_query(self, query: str) -> str:
        """
        使用大模型处理查询并调用可用的 MCP 工具 (Function Calling)
        """
        messages = [{"role": "user", "content": query}]
        
        response = await self.session.list_tools()
        
        available_tools = [{
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "input_schema": tool.inputSchema
            }
        } for tool in response.tools]
        # print(available_tools)
        
        response = self.client.chat.completions.create(
            model=self.model,            
            messages=messages,
            tools=available_tools     
        )
        
        # 处理返回的内容
        content = response.choices[0]
        if content.finish_reason == "tool_calls":
            # 如何是需要使用工具，就解析工具
            tool_call = content.message.tool_calls[0]
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            # 执行工具
            result = await self.session.call_tool(tool_name, tool_args)
            print(f"\n\n[Calling tool {tool_name} with args {tool_args}]\n\n")
            
            # 将模型返回的调用哪个工具数据和工具执行完成后的数据都存入messages中
            messages.append(content.message.model_dump())
            messages.append({
                "role": "tool",
                "content": result.content[0].text,
                "tool_call_id": tool_call.id,
            })
            
            # 将上面的结果再返回给大模型用于生产最终的结果
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
            )
            return response.choices[0].message.content
            
        return content.message.content
    
    async def chat_loop(self):
        """运行交互式聊天循环"""
        print("\n🤖 MCP 客户端已启动！输入 'quit' 退出")

        while True:
            try:
                query = input("\n你: ").strip()
                if query.lower() == 'quit':
                    break
                
                response = await self.process_query(query)  # 发送用户输入到 OpenAI API
                print(f"\n🤖 OpenAI: {response}")

            except Exception as e:
                print(f"\n⚠️ 发生错误: {str(e)}")

    async def cleanup(self):
        """清理资源"""
        await self.exit_stack.aclose()

async def main():
    if len(sys.argv) < 2:
        print("Usage: python client.py <path_to_server_script>")
        sys.exit(1)

    client = MCPClient()
    try:
        await client.connect_to_server(sys.argv[1])
        await client.chat_loop()
    finally:
        await client.cleanup()

if __name__ == "__main__":
    import sys
    asyncio.run(main())
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1742291307705.jpg" alt="1742291307705" style="zoom:33%;" />

- 创建.env文件

&emsp;&emsp;接下来继续创建一个`.env`文件，来保存大模型调用的API-KEY

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202503171539087.png" alt="image-20250317153902986" style="zoom:50%;" />

并写入如下内容：

```bash
BASE_URL=https://api.deepseek.com
MODEL=deepseek-chat
OPENAI_API_KEY=YOUR_DEEPSEEK_API_KEY
```

此时项目结构如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318174756635.png" alt="image-20250318174756635" style="zoom:50%;" />

- 运行MCP客户端+服务器

&emsp;&emsp;最后在命令行中执行如下命令，即可开启对话：

```bash
uv run client.py server.py
```

直接提问`请问北京今天天气如何？`运行结果如下所示：

<img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318175419960.png" alt="image-20250318175419960" style="zoom:50%;" />

至此，即完成了一次简单的MCP执行流程。

### 4.MCP+Agents SDK基础调用流程

&emsp;&emsp;而在新版的Agents SDK中，Agents SDK可以将某个对应的Agent封装为client与外部定义好的server进行通信。基本实现流程如下，还是查询天气的server.py，现在将其复制到jupyter运行主目录下，并修改名称为`weather_server.py`：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403173507052.png" alt="image-20250403173507052" style="zoom:33%;" />

然后导入相关的库：

In [9]:
import asyncio
import os
import shutil
import subprocess
import time
from typing import Any

from agents import Agent, Runner, gen_trace_id, trace
from agents.mcp import MCPServer, MCPServerStdio
from agents.model_settings import ModelSettings

同时定义Agent+MCP运行函数，要求带入MCPServer对象，且带入mcp_servers中，作为类似tools参数带入到当前Agent运行过程中：

In [44]:
async def run(mcp_server: MCPServer):
    
    agent = Agent(
        name="Assistant",
        instructions="你是一名助人为乐的助手",
        mcp_servers=[mcp_server],
        model=deepseek_model
    )

    message = "请帮我查询北京今天天气如何？"
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)

然后创建mcp_run函数，负责开启外部server并运行Agent：

In [45]:
async def mcp_run():
    async with MCPServerStdio(
        name="Weather Server",
        cache_tools_list=True,
        params = {"command": "uv","args": ["run", "weather_server.py"]} 
    ) as server:
        await run(server)

✅ 关键组件解释：

- `async with MCPServerStdio(...) as server:`  
  启动一个 MCP 工具服务器进程，使用标准输入输出（`stdio`）作为通信协议，并在上下文中运行（退出时会自动关闭）。

- `name="Weather Server"`  
  给这个 MCP Server 起名为“天气服务器”，这只是用于日志和识别用的标识符。

- `cache_tools_list=True`  
  意思是：首次加载工具时缓存工具列表，后续不需要重新请求工具元数据（提升效率）。

- `params = {"command": "uv", "args": ["run", "weather_server.py"]}`  
  这是启动 MCP 工具服务器的 **命令行参数**，等价于在命令行里运行：
  ```bash
  uv run weather_server.py
  ```
  - `uv`：同样使用[uv](https://github.com/astral-sh/uv)，是一个快速的 Python 包和环境管理器，比 pip/venv 更高效。
  - `run weather_server.py`：运行一个你定义好的MCP server。


最后测试运行：

In [46]:
await mcp_run()

Running: 请帮我查询北京今天天气如何？
北京的今天天气如下：

🌍 北京, 中国  
🌡 温度: 18.94°C  
💧 湿度: 15%  
🌬 风速: 4.52 m/s  
🌤 天气: 多云  

天气较为凉爽，适合外出活动哦！


## 二、Agents SDK+MCP进阶技术实践

### 1.Agents SDK接入更多开源MCP服务器流程

&emsp;&emsp;若要采用MCP技术栈，最核心的便利就在于可以快速接入海量MCP开源服务器，无需反复开发，即可快速丰富当前Agent功能。热门MCP server合集地址如下：

- Model Context Protocol servers: https://github.com/modelcontextprotocol/servers
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403174645233.png" alt="image-20250403174645233" style="zoom:33%;" />

- Awesome MCP Servers: https://github.com/punkpeye/awesome-mcp-servers
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403174514932.png" alt="image-20250403174514932" style="zoom:33%;" />

- MCP导航： https://mcp.so/
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403174757712.png" alt="image-20250403174757712" style="zoom:50%;" /></center>

接下来我们就先尝试接入一个开源的、开发人员常用的MCP服务器——mcp-server-git。

- mcp-server-git：https://github.com/modelcontextprotocol/servers/tree/main/src/git
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403175004931.png" alt="image-20250403175004931" style="zoom:33%;" />

##### 📌 MCP-Server-Git 简介

`mcp-server-git` 是一个遵循 **Model Context Protocol (MCP)** 的 Git 操作服务器，专为大语言模型与 Git 仓库的交互而设计。通过该服务，模型可以安全、结构化地完成 Git 操作，包括状态查询、差异比较、提交更改、分支管理等，从而实现自动化代码管理与协作。


✨ 核心功能包括：

- **查询仓库状态**：获取当前工作区和暂存区的变动情况（`git_status`、`git_diff_unstaged`、`git_diff_staged`）
- **版本比较**：支持分支或提交之间的差异查看（`git_diff`）
- **代码提交与暂存管理**：支持新增、暂存、撤销暂存、更改提交（`git_add`、`git_reset`、`git_commit`）
- **日志查询与历史查看**：获取提交历史、查看具体提交内容（`git_log`、`git_show`）
- **分支操作**：新建分支、切换分支（`git_create_branch`、`git_checkout`）
- **仓库初始化**：支持新建空 Git 仓库（`git_init`）

##### 🚀 调用方式
✅ 使用 `uvenv` 快速启动（推荐方式）

无需安装，只需一行命令即可运行：

```bash
uvenv run mcp-server-git
```

首次运行会从 PyPI 下载并缓存，后续启动速度更快。


📡 接口调用格式（示例）

以 `git_status` 为例，MCP 工具调用格式如下：

```json
{
  "tool": "git_status",
  "input": {
    "repo_path": "/path/to/your/git/repo"
  }
}
```

服务将返回 Git 工作目录当前状态的文本描述。

- uvx（已弃用）/ uvenv 工具安装与使用指南（Ubuntu）

`uvx`（现已由 `uvenv` 取代）是一个基于 Python 的零安装运行工具，可快速运行 Python 包，而无需手动安装或污染系统环境。

1. uvenv 安装方法（Ubuntu）

打开终端，运行以下命令自动安装最新版 `uv`（含 `uvenv`）：

```bash
curl -Ls https://astral.sh/uv/install.sh | sh
```

或使用 `wget` 命令安装：

```bash
wget -qO- https://astral.sh/uv/install.sh | sh
```

安装完成后，验证是否安装成功：

```bash
uv --version
uvenv --help
```

🔁 可选：通过 pip 安装（不推荐）

```bash
pip install uv
```

虽然可以使用 pip 安装 `uv`，但官方推荐使用上面的安装脚本，因为：
- 安装的是预编译二进制版本，性能更好
- 功能更完整
- 自动将可执行文件添加到环境路径


2. uvenv 功能说明

- **无需提前安装**：`uvenv` 自动从 PyPI 拉取并运行 Python 包。
- **隔离环境**：每次执行均为临时环境，避免污染系统全局环境。
- **缓存机制**：首次拉取后自动缓存，后续使用直接读取缓存，提高运行速度。
- **用途广泛**：适用于快速测试、命令行工具临时使用以及 Python 脚本的隔离执行。


3. uvenv 基本使用方法

使用语法

```bash
uvenv run <package-name>
```

示例：运行 `mcp-server-git`

首次运行会自动从 PyPI 下载并缓存：

```bash
uvenv run mcp-server-git
```

再次运行则直接使用缓存，速度更快：

```bash
uvenv run mcp-server-git
```

⚠️ 更新缓存中的包

默认情况下，`uvenv` 不会自动检测和使用 PyPI 上的新版本。如果想强制使用最新版本，可以：

- 清理缓存后重新运行：

```bash
uv cache clear
uvenv run mcp-server-git
```


4. 常用操作

- **查看缓存位置**：

```bash
ls ~/.cache/uv
```

- **清理缓存**（若需要）：

```bash
uv cache clear
```

- **升级 uv/uvenv 工具**：

```bash
uv self update
```

- **从 uvx 迁移提示**：如果你看到如下提示：

```bash
Deprecation Message: `uvx` is deprecated in favor of `uvenv`.
```
说明你已成功迁移到新版，请使用 `uvenv run <package>` 来代替原来的 `uvx <package>` 命令。

&emsp;&emsp;接下来尝试在Jupyter中创建Agent来完成调用：

In [81]:
async def run(mcp_server: MCPServer, directory_path: str):
    
    agent = Agent(
        name="Assistant",
        instructions=f"Answer questions about the git repository at {directory_path}, use that for repo_path",
        mcp_servers=[mcp_server],
        model=deepseek_model
    )

    message = "请帮我介绍下这个项目。"
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)

In [87]:
async def mcp_run(directory_path):
    async with MCPServerStdio(
        name="GitHub Server",
        cache_tools_list=True,
        params={"command": "uvenv", "args": ["run", "mcp-server-git"]},
    ) as server:
        await run(server, directory_path=directory_path)

接下来我们查询DeepSeek大模型部署推理加速库Ktransformers的新版本特性：

In [88]:
directory_path = 'https://github.com/kvcache-ai/ktransformers'

In [89]:
# 可选，设置代理
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10080'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10080'

# 取消代理设置
# os.environ.pop('HTTP_PROXY', None)
# os.environ.pop('HTTPS_PROXY', None)

# AutoDL学术加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

⚠需要注意的是，mcp-server-git 是一个持续运行的 CLI 工具服务，而受限于jupyter环境，以下代码运行结束后并不会自动退出，需要重启jupyter-kernel才能关闭进程，请谨慎运行：

In [ ]:
await mcp_run(directory_path)

Running: 请帮我介绍下这个项目。
目前无法直接获取该项目的提交日志或详细信息。如果您能提供更具体的需求或问题，我可以帮助您进一步了解该项目。例如：

1. **项目功能**：您想了解该项目的核心功能或用途吗？
2. **代码结构**：是否需要查看项目的文件结构或某些文件的内容？
3. **贡献者**：是否需要了解项目的贡献者或活跃度？
4. **其他信息**：是否有其他特定的信息需要查询？

请告诉我您的具体需求，我将尽力协助！


而一种更加稳妥的方式，是定义py脚本并在命令行中运行：

```python
import asyncio
import shutil

from agents import Agent, Runner, trace
from agents.mcp import MCPServer, MCPServerStdio


async def run(mcp_server: MCPServer, directory_path: str):
    agent = Agent(
        name="Assistant",
        instructions=f"Answer questions about the git repository at {directory_path}, use that for repo_path",
        mcp_servers=[mcp_server],
        model=deepseek_model
    )

    message = "请帮我介绍下这个项目。"
    print("\n" + "-" * 40)
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)


async def main():
    async with MCPServerStdio(
        cache_tools_list=True, 
        params={"command": "uvenv", "args": ["run", "mcp-server-git"]},
    ) as server:
        await run(server, directory_path)


if __name__ == "__main__":
    if not shutil.which("uvenv"):
        raise RuntimeError("uvx is not installed. Please install it with `pip install uvx`.")

    asyncio.run(main())
```

### 2.Agents SDK接入多个MCP服务器流程

&emsp;&emsp;接下来我们继续介绍如何将Agents SDK同时接入多个MCP服务器，理论上，MCP一个服务器能同时运行多个外部函数，而一个MCP Client则可以连接多个MCP服务器：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403183654681.png" alt="image-20250403183654681" style="zoom:33%;" />

而Agents SDK本身也是可以作为MCP Client的，因此是完全可以连接多个MCP server，其基本实现函数如下：

In [9]:
async def mcp_run_multi(servers_params, message):
    # 使用 AsyncExitStack 自动管理多个上下文退出
    async with AsyncExitStack() as stack:
        servers = []
        # 创建并进入所有 server 上下文
        for p in servers_params:
            server = MCPServerStdio(
                name=p.get("name", "Unnamed Server"),
                cache_tools_list=True,
                params={
                    "command": "uv",
                    "args": ["run", p["script"]],
                },
            )
            entered_server = await stack.enter_async_context(server)
            servers.append(entered_server)
        
        # 构造 agent，传入多个 server
        agent = Agent(
            name="Assistant",
            instructions="你是一名助人为乐的助手",
            mcp_servers=servers,
            model_settings=ModelSettings(tool_choice="required"),
            model=deepseek_model
        )
        
        print(f"Running: {message}")
        result = await Runner.run(starting_agent=agent, input=message)
        print(result.final_output)

        return result

这里我们尝试创建一个“写入本地文档”的MCP服务器：`weather_server.py`，代码如下：

```python
import json
import httpx
from typing import Any
from mcp.server.fastmcp import FastMCP

# 初始化 MCP 服务器
mcp = FastMCP("WriteServer")
USER_AGENT = "write-app/1.0"

@mcp.tool()
async def write_file(content: str) -> str:
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return：是否成功写入
    """
    return "已成功写入本地文件。"

if __name__ == "__main__":
    # 以标准 I/O 方式运行 MCP 服务器
    mcp.run(transport='stdio')
```

并将其放在Jupyter主目录下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403184029332.png" alt="image-20250403184029332" style="zoom:33%;" />

然后尝试同时调用多个server，结果如下：

In [24]:
# 示例调用：传入多个 server 的配置
result = await mcp_run_multi(
    servers_params=[
        {"name": "Weather Server", "script": "weather_server.py"},
        {"name": "Writer Server", "script": "write_server.py"}
    ],
    message="请帮我查询北京天气，并写入本地文档。"
)

Running: 请帮我查询北京天气，并写入本地文档。
北京的天气信息已查询并成功写入本地文档：

🌍 Beijing, CN  
🌡 温度: 19.94°C  
💧 湿度: 13%  
🌬 风速: 5.51 m/s  
🌤 天气: 多云  


In [27]:
len(result.new_items)

5

In [29]:
result.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"city":"Beijing"}', call_id='call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f', name='query_weather', type='function_call', id='__fake_id__', status=None)

In [30]:
result.new_items[1].raw_item

{'call_id': 'call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f',
 'output': '{"type":"text","text":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云\\n","annotations":null}',
 'type': 'function_call_output'}

In [31]:
result.new_items[2].raw_item

ResponseFunctionToolCall(arguments='{"content":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云"}', call_id='call_0_e137b744-d34a-4dba-8758-456c2aca5eb0', name='write_file', type='function_call', id='__fake_id__', status=None)

In [32]:
result.new_items[3].raw_item

{'call_id': 'call_0_e137b744-d34a-4dba-8758-456c2aca5eb0',
 'output': '{"type":"text","text":"已成功写入本地文件。","annotations":null}',
 'type': 'function_call_output'}

In [33]:
result.new_items[4].raw_item

ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='北京的天气信息已查询并成功写入本地文档：\n\n🌍 Beijing, CN  \n🌡 温度: 19.94°C  \n💧 湿度: 13%  \n🌬 风速: 5.51 m/s  \n🌤 天气: 多云  ', type='output_text')], role='assistant', status='completed', type='message')

In [34]:
result.to_input_list()

[{'content': '请帮我查询北京天气，并写入本地文档。', 'role': 'user'},
 {'arguments': '{"city":"Beijing"}',
  'call_id': 'call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f',
  'name': 'query_weather',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f',
  'output': '{"type":"text","text":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云\\n","annotations":null}',
  'type': 'function_call_output'},
 {'arguments': '{"content":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云"}',
  'call_id': 'call_0_e137b744-d34a-4dba-8758-456c2aca5eb0',
  'name': 'write_file',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_0_e137b744-d34a-4dba-8758-456c2aca5eb0',
  'output': '{"type":"text","text":"已成功写入本地文件。","annotations":null}',
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': '北京的天气信息已查询并成功写入本地文档：\n\n🌍 Beijing, CN  \n🌡 温度: 19.94°C  \n💧 湿度: 13%  \n🌬 风

不难看出，Agents SDK对于MCP实现过程基本遵照Function calling来执行。

In [15]:
# 示例调用：传入多个 server 的配置
result = await mcp_run_multi(
    servers_params=[
        {"name": "Weather Server", "script": "weather_server.py"},
        {"name": "Writer Server", "script": "write_server.py"}
    ],
    message="请帮我查询北京和杭州天气，并写入本地文档。"
)

Running: 请帮我查询北京和杭州天气，并写入本地文档。
已成功查询北京和杭州的天气信息，并写入本地文档。以下是天气详情：

### 北京天气
- **温度**: 17.94°C
- **湿度**: 16%
- **风速**: 3.24 m/s
- **天气**: 多云

### 杭州天气
- **温度**: 14.95°C
- **湿度**: 40%
- **风速**: 3.22 m/s
- **天气**: 晴

文件已保存。


In [16]:
len(result.new_items)

7

In [17]:
result.to_input_list()

[{'content': '请帮我查询北京和杭州天气，并写入本地文档。', 'role': 'user'},
 {'arguments': '{"city": "Beijing"}',
  'call_id': 'call_0_fe02f63b-7a08-4983-8098-6e50821100ac',
  'name': 'query_weather',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'arguments': '{"city": "Hangzhou"}',
  'call_id': 'call_1_98d66c45-6268-42b5-bc0b-15a4cd857ece',
  'name': 'query_weather',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_0_fe02f63b-7a08-4983-8098-6e50821100ac',
  'output': '{"type":"text","text":"🌍 Beijing, CN\\n🌡 温度: 17.94°C\\n💧 湿度: 16%\\n🌬 风速: 3.24 m/s\\n🌤 天气: 多云\\n","annotations":null}',
  'type': 'function_call_output'},
 {'call_id': 'call_1_98d66c45-6268-42b5-bc0b-15a4cd857ece',
  'output': '{"type":"text","text":"🌍 Hangzhou, CN\\n🌡 温度: 14.95°C\\n💧 湿度: 40%\\n🌬 风速: 3.22 m/s\\n🌤 天气: 晴\\n","annotations":null}',
  'type': 'function_call_output'},
 {'arguments': '{"content":"🌍 Beijing, CN\\n🌡 温度: 17.94°C\\n💧 湿度: 16%\\n🌬 风速: 3.24 m/s\\n🌤 天气: 多云\\n\\n🌍 Hangzhou, CN\\n🌡 温度: 14.95°C\\n💧 

整体实现流程和上一小节介绍的Agents SDK Function calling执行流程完全一致：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250401162048526.png" alt="image-20250401162048526" style="zoom:50%;" />

## 三、Agents SDK搭建Agentic RAG（1）：GraphRAG API调用与外部函数封装

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403193119540.png" alt="image-20250403193119540" style="zoom:50%;" />

#### 1.GraphRAG安装与项目创建

- **Step 1.使用pip安装graphrag**

```bash
pip install graphrag
```

Jupyter中可以使用如下方式进行安装：

In [1]:
pip install graphrag

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.2/365.2 kB 4.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.2/303.2 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 13.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.2/30.2 MB 13.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 12.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 

需要注意的是，本次公开课采用GraphRAG最新版 2.1.0进行教学

In [87]:
pip show graphrag

Name: graphrag
Version: 2.1.0
Summary: GraphRAG: A graph-based retrieval-augmented generation (RAG) system.
Home-page: 
Author: Alonso Guevara Fernández
Author-email: alonsog@microsoft.com
License: MIT
Location: /root/miniconda3/lib/python3.12/site-packages
Requires: aiofiles, azure-cosmos, azure-identity, azure-search-documents, azure-storage-blob, devtools, environs, fnllm, future, graspologic, json-repair, lancedb, networkx, nltk, numpy, openai, pandas, pyarrow, pydantic, python-dotenv, pyyaml, rich, spacy, textblob, tiktoken, tqdm, typer, typing-extensions, umap-learn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


最新版GraphRAG特性如下：

1. 架构升级与知识图谱构建优化

- **智能化的知识图谱构建**  
  GraphRAG 2.0 在数据预处理阶段，通过大模型自动抽取文本中的实体及关系，构建出层次化的知识图谱。相比传统的简单文本片段检索，构建后的图谱能够以“社区”（topic-based clusters）的方式对数据进行组织，这样不仅可以覆盖全局信息，也能针对局部查询给出更精准的答案。  

- **动态社区选择机制**  
  新版本引入了动态社区选择流程。系统会在生成响应之前，对知识图谱中不同“社区”的相关性进行评估，从而仅保留与当前查询最匹配的部分。这种机制能有效“丢弃”噪声数据，提高检索效率和答案的准确性。  

---

2. 查询流程与成本优化

- **两阶段查询流程**  
  GraphRAG 2.0 将整个流程拆分为“索引阶段”和“查询阶段”：  
  - 在索引阶段，系统利用大模型对原始数据进行结构化处理，提取实体及其关系，构建分层知识图谱；  
  - 在查询阶段，则先进行初步的相关性测试，再利用经过动态社区筛选的信息来生成上下文丰富、精准的回答。  
  这种分步处理不仅提高了检索的广度和深度，还能根据查询需求灵活调用大模型。  

- **Token消耗大幅降低**  
  为了应对大规模数据调用时高昂的成本，GraphRAG 2.0 对 LLM 的调用做了优化。据报道，在某些场景下整体 Token 消耗降低高达 77%，这使得系统在保证高质量回答的同时，也大幅提升了成本效率。  

- **LazyGraphRAG 模式**  
  新版本还推出了“LazyGraphRAG”模式——一种结合了向量检索和图结构检索优势的方案。该模式采用迭代深化的方式，只有在必要时才调用资源密集型的大模型进行深度分析，从而实现了与传统 GraphRAG 相比成本更低但效果相当的目标。  

---

3. 搜索结果质量与应用扩展

- **精准且上下文丰富的答案**  
  利用层次化的知识图谱和动态社区筛选，GraphRAG 2.0 能够生成更具有可解释性和上下文关联的答案。无论是全局性问题（例如“核心主题是什么？”）还是局部性查询（如“谁、何时、何地”等），系统都能根据不同场景调整检索策略，返回最相关的信息。  

- **多系统和多场景集成**  
  此外，新版本在设计上也更注重与其它系统的集成能力，例如在数字营销场景中，通过与 URL 缩短服务、链接分析等工具相结合，为用户提供定制化且综合的查询反馈。  

- **Step 2.创建检索项目文件夹**

```bash
mkdir -p ./ragtest/input
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319205109586.png" alt="image-20250319205109586" style="zoom:30%;" />

- **Step 3.上传数据集**

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20241128161523454.png" alt="image-20241128161523454" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319205204587.png" alt="image-20250319205204587" style="zoom:30%;" />

- **Step 4.初始化项目文件**

```bash
graphrag init --root ./ragtest
```

In [3]:
!graphrag init --root ./ragtest

Initializing project at /root/autodl-tmp/MCP/ragtest
⠋ GraphRAG Indexer 

- **Step 5.修改项目配置**

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319205316078.png" alt="image-20250319205316078" style="zoom:30%;" />

打开.env文件，填写DeepSeek API-KEY或OpenAI API-Key

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319205455014.png" alt="image-20250319205455014" style="zoom:30%;" />

打开setting.yaml文件，填写模型名称和反向代理地址：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250320175421785.png" alt="image-20250320175421785" style="zoom:33%;" />

#### 2.GraphRAG索引Indexing过程执行

&emsp;&emsp;一切准备就绪后，即可开始执行GraphRAG索引过程。

- **Step 7.借助GraphRAG脚本自动执行indexing**

In [5]:
!graphrag index --root ./ragtest


Logging enabled at /root/autodl-tmp/MCP/ragtest/logs/indexing-engine.log
🚀 LLM Config Params Validated
🚀 Embedding LLM Config Params Validated
Running standard indexing.
⠹ GraphRAG Indexer 
⠸ GraphRAG Indexer e.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠸ GraphRAG Indexer e.text) - 1 files loaded (0 filtered) ━ 100% … 0…
🚀 create_base_text_unitses loaded (0 filtered) ━ 100% … 0…
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
                                                  id  ... n_tokens 0:00:00
0  4ba5acd798a419708cfe5cc86f1fcb6df907a20b1992ad...  ...     1200
1  7a8436d7970da4b0e5c837b0a3eaaa196e6e2284c7c754...  ...     1200
2  1141affdc56bf8f5092b950b57febdedb884673ec6b23d...  ...     1200
3  b5df95b6ec7e3aaea3f67db4914e1b2e874f3ac77fc3c8...  ...      558

[4 rows x 4 columns]
⠼ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠼ GraphRAG Indexer ━━━━━━━━━━━━━━━━━━━━━━━━━ 

该命令也可以在终端内运行，结果如图：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250108152705197.png" alt="image-20250108152705197" style="zoom:33%;" />

运行结束后，各知识图谱相关数据集都保存在output文件夹中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319210041485.png" alt="image-20250319210041485" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319210119624.png" alt="image-20250319210119624" style="zoom:33%;" />

In [7]:
!graphrag query --root ./ragtest --method local --query "请帮我介绍下ID3算法"




INFO: Vector Store Args: {
    "default_vector_store": {
        "type": "lancedb",
        "db_uri": "/root/autodl-tmp/MCP/ragtest/output/lancedb",
        "url": null,
        "audience": null,
        "container_name": "==== REDACTED ====",
        "database_name": null,
        "overwrite": true
    }
}

SUCCESS: Local Search Response:
# ID3 Algorithm Overview

The ID3 (Iterative Dichotomiser 3) algorithm is a classic decision tree algorithm primarily used for classification tasks. It is known for its ability to handle discrete variables and is foundational in the field of machine learning. The algorithm employs information entropy as an evaluation metric and uses information gain to split datasets based on categorical features. However, one of the limitations of ID3 is its tendency to overfit, as it often prefers features with many categories [Data: Entities (0); Reports (1)].

## Key Features and Functionality

ID3 is designed to work with discrete variables, which means it ca

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250108154120541.png" alt="image-20250108154120541" style="zoom:33%;" />

Jupyter中可以直接输入如下内容进行Query：

In [8]:
!graphrag query --root ./ragtest --method local --query "请帮我介绍下ID3算法，请用中文回答。"




INFO: Vector Store Args: {
    "default_vector_store": {
        "type": "lancedb",
        "db_uri": "/root/autodl-tmp/MCP/ragtest/output/lancedb",
        "url": null,
        "audience": null,
        "container_name": "==== REDACTED ====",
        "database_name": null,
        "overwrite": true
    }
}

SUCCESS: Local Search Response:
### ID3算法简介

ID3算法是一种经典的决策树算法，主要用于分类问题。它专注于离散变量，通过信息熵作为评估指标，利用信息增益来基于分类特征对数据集进行划分。ID3算法的一个显著特点是它倾向于选择具有较多类别的特征进行划分，这可能导致过拟合的问题 [Data: Reports (1); Entities (0)]。

### 算法的基本原理

ID3算法的基本原理是通过计算信息增益来选择最佳的特征进行数据集的划分。信息增益是通过计算父节点和子节点的信息熵差值来获得的。信息熵用于衡量数据集的纯度，信息增益则表示在某一特征的基础上划分数据集后纯度的提升 [Data: Sources (0)]。

### 特征选择与数据集划分

在ID3算法中，特征选择是通过计算每个特征的信息增益来实现的。算法会选择信息增益最大的特征进行数据集的划分。例如，特征“年龄”在某些情况下可能提供最大的增益，因此会被优先选择用于划分数据集 [Data: Sources (0); Relationships (4)]。

### ID3的局限性

ID3算法的一个主要局限性是它无法处理连续变量。在处理连续变量时，通常需要先对其进行离散化。此外，ID3算法没有内置的机制来防止过拟合，这使得它在某些情况下可能不如其他算法（如C4.5）鲁棒 [Data: Sources (0); Reports (1)]。

### 总结

ID3算法作为一种基础的决策树算法，为后续算法（如C4.5）的发展奠定了基础。尽管存在一些局限性

#### 3.GraphRAG API 初始化项目

In [10]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

```bash
mkdir -p ./MLRAG/input
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319210839253.png" alt="image-20250319210839253" style="zoom:33%;" />

然后进行初始化

In [134]:
!graphrag init --root ./MLRAG

Initializing project at /root/autodl-tmp/OpenAI体验课/MLRAG
⠋ GraphRAG Indexer 

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319211009912.png" alt="image-20250319211009912" style="zoom:33%;" />

并修改配置文件（同上一部分）。

#### 4.借助API进行Indexing过程

In [11]:
PROJECT_DIRECTORY = "./MLRAG"

生成 GraphRagConfig 对象

In [12]:
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319211249955.png" alt="image-20250319211249955" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250319211354304.png" alt="image-20250319211354304" style="zoom:33%;" />

索引 API


索引是指摄取原始文本数据并构建知识图谱的过程。GraphRAG 目前支持纯文本（.txt）和 .csv 文件格式。

构建索引

In [13]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

[2025-04-03T12:27:09Z WARN  lance::dataset::write::insert] No existing dataset at /root/autodl-tmp/OpenAI体验课/MLRAG/output/lancedb/default-community-full_content.lance, it will be created
[2025-04-03T12:27:12Z WARN  lance::dataset::write::insert] No existing dataset at /root/autodl-tmp/OpenAI体验课/MLRAG/output/lancedb/default-text_unit-text.lance, it will be created
[2025-04-03T12:27:16Z WARN  lance::dataset::write::insert] No existing dataset at /root/autodl-tmp/OpenAI体验课/MLRAG/output/lancedb/default-entity-description.lance, it will be created


index_result 是一个包含索引流水线各个工作流的列表，每个工作流代表一次索引构建过程。

In [14]:
for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

Workflow Name: create_base_text_units	Status: success
Workflow Name: create_final_documents	Status: success
Workflow Name: extract_graph	Status: success
Workflow Name: finalize_graph	Status: success
Workflow Name: create_communities	Status: success
Workflow Name: create_final_text_units	Status: success
Workflow Name: create_community_reports	Status: success
Workflow Name: generate_text_embeddings	Status: success


在此循环中，遍历 index_result 列表，并打印每个工作流的名称及其执行状态。如果工作流中存在错误，则输出错误信息，否则输出 "success"。

#### 5.借助API进行Query过程

查询索引

在查询索引之前，必须先将多个索引文件加载到内存中，并传递给查询 API。

In [16]:
entities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)

上述代码读取 .parquet 格式的索引数据，包括 实体（entities）、社群（communities） 以及 社群报告（community_reports）。

执行全局搜索

In [17]:
query = "请帮我对比下ID3和C4.5决策树算法优劣势。并用中文进行回答。"

In [18]:
response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query=query,
)

在这里，我们调用 `global_search` 方法，使用已加载的索引数据进行查询。  
- `community_level=2`：设定社群层级为 2 级。  
- `dynamic_community_selection=False`：禁用动态社群选择。  
- `response_type="Multiple Paragraphs"`：设置返回的查询结果为多段落格式。  
- `query=query`：查询问题为 **"请帮我对比下ID3和C4.5决策树算法优劣势。并用中文进行回答。"**。

**解析查询结果**  

In [19]:
print(response)

在决策树算法中，ID3和C4.5算法各有其特点和应用场景。以下是对这两种算法优劣势的对比分析：

### ID3算法

ID3算法是决策树算法的基础，主要特点是使用信息增益作为选择特征的标准来构建决策树，这使得算法简单直观，易于理解和实现。它适用于处理分类问题，特别是在特征都是离散值的情况下表现良好。然而，ID3算法也存在一些明显的劣势，包括无法直接处理连续属性和缺乏剪枝处理，这可能导致模型容易过拟合，影响模型的泛化能力 [Data: Reports (3)]。

### C4.5算法

C4.5算法在ID3的基础上进行了改进，引入了信息增益比来选择特征，这一改进不仅使得算法能够处理连续属性，还加入了剪枝策略，有效减少了过拟合的风险。因此，C4.5算法的模型具有更好的泛化能力。尽管C4.5算法的计算复杂度相对ID3算法有所增加，但其能够处理更广泛的数据类型和更复杂的分类问题，使其成为更加强大和灵活的决策树算法 [Data: Reports (3)]。

### 综合对比

总的来说，ID3算法以其简单直观的特性适用于特征全为离散值的分类问题，但在处理连续属性和防止过拟合方面存在局限。而C4.5算法通过引入信息增益比和剪枝策略，不仅克服了ID3算法的这些限制，还提高了模型的泛化能力，使其能够更有效地处理各种类型的数据和复杂的分类问题。因此，在选择决策树算法时，应根据具体的数据特征和需求来决定使用ID3算法还是C4.5算法。


`response` 变量是 **GraphRAG** 返回的正式查询结果。  

In [20]:
pprint(context)

{'reports':     id                                              title  occurrence weight  \
0    2            CART and Its Impact on Machine Learning           1.000000   
1    7  Scikit-Learn Ecosystem and Its Integration wit...           1.000000   
2    3  Evolution of Decision Tree Algorithms: From ID...           0.888889   
3   18  Scikit-Learn: A Comprehensive Machine Learning...           0.888889   
4   11            CART and Its Impact on Machine Learning           0.666667   
5    6               Scikit-Learn Decision Tree Ecosystem           0.666667   
6   12  Advancements in Machine Learning: The Friedman...           0.555556   
7    0        Scikit-Learn and Machine Learning Community           0.444444   
8   16  Scikit-Learn's Core Components for Machine Lea...           0.444444   
9    8  Scikit-Learn Ecosystem: A Comprehensive Machin...           0.444444   
10   4       Decision Tree Algorithms in Machine Learning           0.333333   
11   1  Decision Tree Parame

`context` 变量包含关于查询过程的详细元数据，包括：
- 查询过程中检索到的数据信息
- 被用于构建上下文的文本片段
- 其他元数据  

深入分析 `context` 对象可以获取更精细的信息，比如**LLM 模型最终使用的文本数据来源**。

#### 6.封装函数完成GraphRAG Query

In [21]:
@function_tool
async def rag_ML(query: str) -> str:
    """
    输入ID3和C4.5决策树相关问题，可以获得精准答案。
    :param query: 机器学习领域ID3和C4.5决策树的相关问题
    :return: query问题对应的答案
    """
    PROJECT_DIRECTORY = "./MLRAG"
    graphrag_config = load_config(Path(PROJECT_DIRECTORY))
    
    # 加载实体
    entities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
    # 加载社区
    communities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
    # 加载社区报告
    community_reports = pd.read_parquet(
        f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
    )
    # 进行全局搜索
    response, context = await api.global_search(
        config=graphrag_config,
        entities=entities,
        communities=communities,
        community_reports=community_reports,
        community_level=2,
        dynamic_community_selection=False,
        response_type="Multiple Paragraphs",
        query=query,
    )
    
    return response

In [ ]:
query = '请帮我对比下ID3和C4.5决策树算法优劣势。并用中文进行回答。'

In [ ]:
await rag_ML(query)

In [25]:
query

'请帮我对比下ID3和C4.5决策树算法优劣势。并用中文进行回答。'

In [26]:
rag_ML(query)

'### ID3算法与C4.5算法的对比\n\n#### ID3算法概述\n\nID3算法是由Ross Quinlan开发的决策树算法的早期版本，它主要通过信息增益（Information Gain）作为选择属性的标准，专注于处理分类问题。ID3算法的主要优点在于其简单直观，易于理解和实现，使其成为决策树算法中的一个重要基石 [Data: Reports (1, 13)]。然而，ID3算法存在几个显著的限制，包括无法直接处理连续属性和缺失值，以及缺乏有效的过拟合预防措施，这些限制可能会影响算法的泛化能力和应用范围 [Data: Reports (1, 13)]。\n\n#### C4.5算法改进\n\nC4.5算法是在ID3的基础上进行改进的，同样由Ross Quinlan开发。C4.5算法的主要改进包括引入信息增益率（Gain Ratio）来选择属性，这一改进解决了ID3算法在属性选择时偏向于选择取值多的属性的问题。此外，C4.5算法能够处理连续属性和缺失值，并引入了剪枝技术来减少过拟合的风险，从而提高了模型的泛化能力 [Data: Reports (1, 13)]。C4.5算法的这些改进使得它更加健壮，适用于更广泛的数据集。然而，需要注意的是，C4.5算法的计算复杂度较高，尤其是在处理大型数据集时，这可能会成为其应用的一个限制 [Data: Reports (1, 13)]。\n\n#### 总结\n\n总的来说，C4.5算法在功能上对ID3算法进行了显著的改进，包括处理连续属性和缺失值的能力，以及引入剪枝策略来避免过拟合问题，这些改进显著提高了决策树模型的泛化能力。然而，这些改进也带来了更高的计算复杂度，特别是在处理大规模数据集时 [Data: Reports (1, 13)]。因此，在选择使用ID3算法还是C4.5算法时，需要根据具体的应用场景和数据集的特点来权衡算法的优势和潜在的限制。'

- 机器学习知识检索智能体

In [22]:
RAG_agent = Agent(
    name="机器学习知识库问答智能体",  
    instructions="可以通过调用rag_ML工具，进行机器学习ID3、C4.5等知识点高精度检索",  
    tools=[rag_ML],
    model=deepseek_model
)

In [23]:
RAG_result = await Runner.run(RAG_agent, "请问ID3的建模流程是什么？")

In [24]:
RAG_result.final_output

'ID3的建模流程主要包括以下步骤：\n\n1. **选择最优特征**：通过计算每个特征的信息增益，选择信息增益最大的特征作为当前节点的分割标准。信息增益越大，表明该特征对分类的贡献越大。\n\n2. **分割数据集**：根据选定的特征将数据集分割成多个子集，每个子集对应特征的一个取值。\n\n3. **递归构建子树**：对每个子集重复上述过程，选择新的最优特征进行分割，直到满足停止条件。\n\n4. **停止条件**：\n   - 所有特征都已使用完毕。\n   - 当前子集中的所有样本属于同一类别。\n   - 没有更多的样本可以分割。\n\n### ID3算法的特点与限制\n- **特点**：简单直观，适合处理离散特征。\n- **限制**：\n  - 无法直接处理连续特征，需要离散化。\n  - 缺乏剪枝机制，容易导致过拟合。\n\n这些限制促使了后续算法（如C4.5）的改进，以更好地适应复杂数据场景。'

## 四、Agents SDK搭建Agentic RAG（2）：联网+知识检索功能实现

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403193119540.png" alt="image-20250403193119540" style="zoom:50%;" />

- 读取全部所需变量

同时，需要在当前项目文件夹内创建一个.env文件，并写入如下变量：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250328202935210.png" alt="image-20250328202935210" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250328203026266.png" alt="image-20250328203026266" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250328203949784.png" alt="image-20250328203949784" style="zoom:50%;" />

- 搜索测试

In [27]:
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10080'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10080'

In [28]:
# 谷歌搜索服务器
google_search_key = os.getenv("GOOGLE_SEARCH_API_KEY")
cse_id = os.getenv("CSE_ID")
search_cookie = os.getenv("search_cookie")
search_ueser_agent = os.getenv("search_ueser_agent")

In [29]:
def google_search(query, num_results=10, site_url=None):
    
    api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
    cse_id = os.getenv("CSE_ID")
    
    url = "https://www.googleapis.com/customsearch/v1"

    # API 请求参数
    if site_url == None:
        params = {
        'q': query,          
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results   
        }
    else:
        params = {
        'q': query,         
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results,  
        'siteSearch': site_url
        }

    # 发送请求
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 解析响应
    search_results = response.json().get('items', [])

    # 提取所需信息
    results = [{
        'title': item['title'],
        'link': item['link'],
        'snippet': item['snippet']
    } for item in search_results]

    return results

In [30]:
results = google_search(query="什么是MCP", num_results=5, site_url='https://www.zhihu.com/')
results

[{'title': '如何评价Anthropic发布的Model Context Protocol (MCP) 协议？ - 知乎',
  'link': 'https://www.zhihu.com/question/5290049088',
  'snippet': 'Nov 27, 2024 ... 轻松愉快的SQL 数据分析，只不过是冰山一角. 协议. Claude (Anthropic) 最近出了个MCP (Model Context Protocol，模型上下文协议) 协议，搞得我朋友圈里\xa0...'},
 {'title': 'MCP是什么？ - 知乎',
  'link': 'https://www.zhihu.com/question/7762420288',
  'snippet': 'Dec 24, 2024 ... 虽然Cursor 已经能自动生成代码，但我们梦想中的AI 编程工具，应该是这样的：. 直接调用AI 查看本地数据库，获取关键信息；; 报错信息一键丢给搜索引擎，秒\xa0...'},
 {'title': 'MCP、function calling 这两者有什么区别？与AI Agent 是什么关系 ...',
  'link': 'https://www.zhihu.com/question/13800647198',
  'snippet': 'Mar 1, 2025 ... 为什么是MCP？ 看到这里你可能有一个问题，在23 年OpenAI 发布GPT function calling 的时候，不是也是可以实现类似的功能吗？我们之前博客介绍的AI Agent，\xa0...'},
 {'title': 'MCP是什么？ - AIQL 的回答- 知乎',
  'link': 'https://www.zhihu.com/question/7762420288/answer/63135932743',
  'snippet': 'Dec 24, 2024 ... 2024年11月底，Anthropic公司发布了全新的MCP（Model Context Protocol）协议，即模型上下文协议。该协议…'},
 {'title': 'AI领域的agent是什么意思？ - NLP自然语言处理的回答- 知乎',
  'link': 

In [31]:
def windows_compatible_name(s, max_length=255):
    """
    将字符串转化为符合Windows文件/文件夹命名规范的名称。
    
    参数:
    - s (str): 输入的字符串。
    - max_length (int): 输出字符串的最大长度，默认为255。
    
    返回:
    - str: 一个可以安全用作Windows文件/文件夹名称的字符串。
    """

    # Windows文件/文件夹名称中不允许的字符列表
    forbidden_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

    # 使用下划线替换不允许的字符
    for char in forbidden_chars:
        s = s.replace(char, '_')

    # 删除尾部的空格或点
    s = s.rstrip(' .')

    # 检查是否存在以下不允许被用于文档名称的关键词，如果有的话则替换为下划线
    reserved_names = ["CON", "PRN", "AUX", "NUL", "COM1", "COM2", "COM3", "COM4", "COM5", "COM6", "COM7", "COM8", "COM9", 
                      "LPT1", "LPT2", "LPT3", "LPT4", "LPT5", "LPT6", "LPT7", "LPT8", "LPT9"]
    if s.upper() in reserved_names:
        s += '_'

    # 如果字符串过长，进行截断
    if len(s) > max_length:
        s = s[:max_length]

    return s

In [32]:
def get_search_text(q, url):
    cookie = os.getenv('search_cookie')
    user_agent = os.getenv('search_ueser_agent')

    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    if title == None:
        return None
    
    else:
        title = windows_compatible_name(title)

        # 创建问题答案正文
        text = ''
        for t in text_d:
            txt = str(t).replace('\n', ' ')
            text += txt

        # 如果有code，则将code追加到正文的追后面
        if code_:
            for c in code_:
                co = str(c).replace('\n', ' ')    
                text += co

        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
        json_data = [
            {
                "link": url,
                "title": title,
                "content": text,
                "tokens": len(encoding.encode(text))
            }
        ]
        
        # 自动创建目录，如果不存在的话
        dir_path = f'./auto_search/{q}'
        os.makedirs(dir_path, exist_ok=True)
    
        with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
            json.dump(json_data, f)

        return title

In [33]:
url = 'https://www.zhihu.com/question/7762420288'
q = "什么是MCP"

In [34]:
get_search_text(q, url)

'MCP是什么？'

In [35]:
def get_search_result(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 默认搜索返回5个答案
    results = google_search(query=q, num_results=5, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            # print(jd[0]['content'])
            content += jd[0]['content']
        else:
            break
    return(content)

In [36]:
get_search_result(q)

'MCP的优势在于：一是开放标准利于服务商开发API，二是避免开发者重复造轮子，可利用现有MCP服务增强Agent。MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。举个栗子，在过去，为了让大模型等 AI 应用使用我们的数据，要么复制粘贴，要么上传下载，非常麻烦。即使是最强大模型也会受到数据隔离的限制，形成信息孤岛，要做出更强大的模型，每个新数据源都需要自己重新定制实现，使真正互联的系统难以扩展，存在很多的局限性。现在，MCP 可以直接在 AI 与数据（包括本地数据和互联网数据）之间架起一座桥梁，通过 MCP 服务器和 MCP 客户端，大家只要都遵循这套协议，就能实现“万物互联”。有了MCP，可以和数据和文件系统、开发工具、Web 和浏览器自动化、生产力和通信、各种社区生态能力全部集成，实现强大的协作工作能力，它的价值远不可估量。这两种技术都旨在增强 AI 模型与外部数据的交互能力，但 MCP 不止可以增强 AI 模型，还可以是其他的应用系统。这样一个理想的“万物互联”生态系统看着很让人着迷。但是大家是不是担心通过 MCP Server 暴露出来的数据会泄露或被非法访问，这个头疼的问题 MCP 也考虑到了。MCP 通过标准化的数据访问接口，大大减少了直接接触敏感数据的环节，降低了数据泄露的风险。还有，MCP 内置了安全机制，确保只有经过验证的请求才能访问特定资源，相当于在数据安全又加上了一道防线。同时，MCP协议还支持多种加密算法，以确保数据在传输过程中的安全性。例如，MCP 服务器自己控制资源，不需要将 API 密钥等敏感信息提供给 LLM 提供商。这样一来，即使 LLM 提供商受到攻击，攻击者也无法获取到这些敏感信息。不过，MCP 这套协议/标准，需要大家一起来共建，这个生态才会繁荣，现在，只是测试阶段，一切才刚刚开始，当然，还会涌现出更多的问题。MCP 协议采用了一种独特的架构设计，它将 LLM 与资源之间的通信划分为三个主要部分：客户

综上，联网搜索外部函数和GitHub搜索外部函数如下：

- 联网搜索外部函数

In [37]:
@function_tool
def get_answer(q):
    """
    联网搜索函数，当用户提出的问题超出你的知识库范畴时，可以利用该函数在知乎上进行搜索，并返回相关答案。
    :param q: 必选参数，和用户问题相关的知乎搜索的关键词，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 默认搜索返回5个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    results = google_search(query=q, num_results=5, site_url='https://zhihu.com/')

    # 判断是否需要自动打开浏览器
    if os.getenv("search_with_broswer") == '1':
        # Edge 浏览器的路径，需要找到你的浏览器地址，假设使用edge浏览器
        edge_path = "C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe"
        # 注册 Edge 浏览器
        webbrowser.register('edge', None, webbrowser.BackgroundBrowser(edge_path))
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        print('正在检索：%s' % url)
        
        # 若打开浏览器，则停留3秒
        if os.getenv("search_with_broswer") == '1':
            webbrowser.get('edge').open(url)
            time.sleep(3)  
            
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            # print(jd[0]['content'])
            content += jd[0]['content']
        else:
            break
    return(content)

> 这里额外设置一个search_with_broswer参数，用于控制是否在搜索的时候同步打开浏览器，以达到如下效果：
> <center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250328203230045.png" alt="image-20250328203230045" style="zoom:50%;" /></center>
> 需要注意的是，无搜索的时候是否同步打开浏览器，都对搜索结果没有影响，只是为了增加一个可视化效果。
> 
> search_with_broswer设置为1时代表打开浏览器，否则代表不打开浏览器。

In [71]:
get_search_result(q="什么是MCP？")

'MCP的优势在于：一是开放标准利于服务商开发API，二是避免开发者重复造轮子，可利用现有MCP服务增强Agent。MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。举个栗子，在过去，为了让大模型等 AI 应用使用我们的数据，要么复制粘贴，要么上传下载，非常麻烦。即使是最强大模型也会受到数据隔离的限制，形成信息孤岛，要做出更强大的模型，每个新数据源都需要自己重新定制实现，使真正互联的系统难以扩展，存在很多的局限性。现在，MCP 可以直接在 AI 与数据（包括本地数据和互联网数据）之间架起一座桥梁，通过 MCP 服务器和 MCP 客户端，大家只要都遵循这套协议，就能实现“万物互联”。有了MCP，可以和数据和文件系统、开发工具、Web 和浏览器自动化、生产力和通信、各种社区生态能力全部集成，实现强大的协作工作能力，它的价值远不可估量。这两种技术都旨在增强 AI 模型与外部数据的交互能力，但 MCP 不止可以增强 AI 模型，还可以是其他的应用系统。这样一个理想的“万物互联”生态系统看着很让人着迷。但是大家是不是担心通过 MCP Server 暴露出来的数据会泄露或被非法访问，这个头疼的问题 MCP 也考虑到了。MCP 通过标准化的数据访问接口，大大减少了直接接触敏感数据的环节，降低了数据泄露的风险。还有，MCP 内置了安全机制，确保只有经过验证的请求才能访问特定资源，相当于在数据安全又加上了一道防线。同时，MCP协议还支持多种加密算法，以确保数据在传输过程中的安全性。例如，MCP 服务器自己控制资源，不需要将 API 密钥等敏感信息提供给 LLM 提供商。这样一来，即使 LLM 提供商受到攻击，攻击者也无法获取到这些敏感信息。不过，MCP 这套协议/标准，需要大家一起来共建，这个生态才会繁荣，现在，只是测试阶段，一切才刚刚开始，当然，还会涌现出更多的问题。MCP 协议采用了一种独特的架构设计，它将 LLM 与资源之间的通信划分为三个主要部分：客户

- 联网检索智能体search_agent创建流程

In [38]:
search_agent = Agent(
    name="联网搜索智能体",  
    instructions="可以执行联网搜索功能，当用户提出的问题超出你的知识范畴时，请先进行搜索，再进行回答。",  
    tools=[get_answer],
    model=deepseek_model
)

In [84]:
search_result = await Runner.run(search_agent, "请问什么是大模型MCP（模型上下文）技术。")

正在接入谷歌搜索，查找和问题相关的答案...
正在检索：https://zhuanlan.zhihu.com/p/27327515233
正在检索：https://zhuanlan.zhihu.com/p/19707405738
正在检索：https://www.zhihu.com/question/5191991346
正在检索：https://www.zhihu.com/question/5290049088
正在检索：https://www.zhihu.com/question/15094843349/answer/130690736046


In [85]:
search_result.final_output

'大模型MCP（Model Context Protocol，模型上下文协议）技术是由Anthropic提出的一种开放标准协议，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信方式。以下是关于MCP技术的详细介绍：\n\n### 1. **MCP的核心目标**\n   - **解决数据孤岛问题**：传统AI模型因数据隔离而难以充分发挥潜力，MCP通过标准化协议连接AI模型与外部数据源，实现数据的安全访问和操作。\n   - **增强模型实用性**：MCP使得AI模型能够动态获取上下文信息，从而提升复杂任务的执行能力。\n\n### 2. **MCP的工作原理**\n   - **客户端-服务器架构**：\n     - **MCP客户端**：作为LLM与MCP服务器之间的桥梁，负责发送请求和处理响应。\n     - **MCP服务器**：提供数据访问和工具调用能力，支持多种功能（如文件操作、数据库查询等）。\n   - **标准化通信协议**：MCP使用JSON-RPC格式进行消息传输，确保通信的标准化和可扩展性。\n\n### 3. **MCP的主要功能**\n   - **资源访问**：支持对本地和远程数据的安全访问，例如文件系统、数据库等。\n   - **工具调用**：允许LLM通过MCP服务器调用外部工具（如Git、GitHub API等）。\n   - **动态提示生成**：MCP服务器可以根据上下文生成动态提示词，优化与LLM的交互。\n\n### 4. **MCP的优势**\n   - **安全性**：通过标准化接口减少敏感数据的直接暴露，支持加密传输和权限控制。\n   - **灵活性**：支持多种数据源和工具，开发者无需重复造轮子。\n   - **生态共建**：MCP是一个开放标准，鼓励社区共同开发和扩展。\n\n### 5. **实际应用场景**\n   - **数据分析**：通过MCP连接SQLite数据库，LLM可以直接查询和分析数据。\n   - **自动化工作流**：结合Git、Slack等工具，实现代码管理和团队协作的自动化。\n   - **多模态扩展**：未来可能支持图像、音频等多模态数据的交互。\n\n### 6. **示例**\n   - **SQLite查询**：用户可以通过自然语言提问（如“查询平均价格”），

- 创建分诊智能体

In [ ]:
triage_agent = Agent(
    name="Triage Agent",  
    instructions="""
    你是一个知识检索智能体，当用户提问机器学习ID3、C4.5相关问题时，请转交给RAG_result进行处理；
    而当用户提出的问题超出你当前知识范畴时，请转交给search_agent进行处理。其他问题请自主进行回答。
    """, 
    handoffs=[RAG_result, search_agent],
    
    model=deepseek_model
)

In [41]:
RAG_agent.handoffs.append(triage_agent)
search_agent.handoffs.append(triage_agent)

- 对话效果测试

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403193119540.png" alt="image-20250403193119540" style="zoom:50%;" />

In [46]:
from agents import (
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

In [47]:
async def chat_assistant():
    
    input_items = []
    current_agent = triage_agent
    
    while True:
        user_input = input("💬 请输入你的消息：")
        if user_input.lower() in ["exit", "quit"]:
            print("✅ 对话已结束")
            break

        input_items.append({"content": user_input, "role": "user"})
        result = await Runner.run(current_agent, input_items)

        for new_item in result.new_items:
            agent_name = new_item.agent.name
            if isinstance(new_item, MessageOutputItem):
                print(f"🧠 {agent_name}: {ItemHelpers.text_message_output(new_item)}")
            elif isinstance(new_item, HandoffOutputItem):
                print(f"🔀 Handed off from {new_item.source_agent.name} to {new_item.target_agent.name}")
            elif isinstance(new_item, ToolCallItem):
                print(f"🔧 {agent_name}: Calling a tool...")
            elif isinstance(new_item, ToolCallOutputItem):
                print(f"📦 {agent_name}: Tool call output: {new_item.output}")
            else:
                print(f"🤷 {agent_name}: Skipping item: {new_item.__class__.__name__}")

        input_items = result.to_input_list()
        current_agent = result.last_agent

In [48]:
await chat_assistant()

💬 请输入你的消息： 你好，请帮我介绍下RNN的基本原理


🧠 Triage Agent: 你好！RNN（Recurrent Neural Network，循环神经网络）是一种专门用于处理序列数据的神经网络模型。它的基本原理是通过引入“循环”结构，使得网络能够保留之前输入的信息，从而对序列数据中的时间依赖性进行建模。以下是RNN的基本原理介绍：

### 1. **循环结构**
RNN的核心特点是其循环结构。与传统的前馈神经网络不同，RNN的隐藏层不仅接收当前时刻的输入，还接收上一时刻的隐藏状态。这种设计使得网络能够记住之前的信息，并将其用于当前时刻的计算。

数学表示：
\[
h_t = f(W_{hx} x_t + W_{hh} h_{t-1} + b_h)
\]
其中：
- \( h_t \) 是当前时刻的隐藏状态。
- \( x_t \) 是当前时刻的输入。
- \( h_{t-1} \) 是上一时刻的隐藏状态。
- \( W_{hx} \) 和 \( W_{hh} \) 是权重矩阵。
- \( b_h \) 是偏置项。
- \( f \) 是激活函数（如tanh或ReLU）。

### 2. **序列处理**
RNN可以处理任意长度的序列数据。对于输入序列 \( x_1, x_2, \ldots, x_T \)，RNN会逐步处理每个时刻的输入，并更新隐藏状态。最终的输出可以基于隐藏状态生成，例如：
\[
y_t = g(W_{yh} h_t + b_y)
\]
其中 \( g \) 是输出层的激活函数（如softmax用于分类任务）。

### 3. **时间依赖性**
由于RNN的隐藏状态传递了之前时刻的信息，因此它能够捕捉序列中的时间依赖性。例如，在自然语言处理中，RNN可以利用前面的单词信息来预测下一个单词。

### 4. **RNN的变体**
标准的RNN存在梯度消失或梯度爆炸的问题，难以捕捉长距离依赖关系。因此，研究者提出了多种改进模型，例如：
- **LSTM（Long Short-Term Memory）**：通过引入门控机制（输入门、遗忘门、输出门）来更好地控制信息的流动。
- **GRU（Gated Recurrent Unit）**：简化版的LSTM，只有更新门和重置门，计算效率更高。

### 5. **应用场景**
RNN广泛应用于以下领域：
- 自然语言处理（NLP）：如语言建模、机器翻

💬 请输入你的消息： 那你知道决策树ID3的建模流程么？


🧠 Triage Agent: 决策树ID3（Iterative Dichotomiser 3）是一种经典的分类算法，主要用于构建决策树模型。其核心思想是通过信息增益来选择最优的特征进行节点分裂，从而递归地构建决策树。以下是ID3算法的建模流程：

---

### **1. 数据准备**
- 输入数据为带有类别标签的训练集，每个样本由多个特征（属性）和一个类别标签组成。
- 所有特征应为离散值（ID3不支持连续值特征，需离散化处理）。

---

### **2. 计算信息熵（Entropy）**
信息熵用于衡量数据的不确定性。对于类别标签 \( C \) 的熵定义为：
\[
\text{Entropy}(S) = -\sum_{i=1}^{k} p_i \log_2 p_i
\]
其中：
- \( S \) 是当前数据集。
- \( p_i \) 是类别 \( i \) 在数据集中的比例。
- \( k \) 是类别的总数。

熵越小，数据的不确定性越低。

---

### **3. 计算信息增益（Information Gain）**
信息增益衡量某个特征对分类任务的贡献。对于特征 \( A \)，其信息增益定义为：
\[
\text{Gain}(S, A) = \text{Entropy}(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \text{Entropy}(S_v)
\]
其中：
- \( \text{Values}(A) \) 是特征 \( A \) 的所有可能取值。
- \( S_v \) 是特征 \( A \) 取值为 \( v \) 的子集。
- \( \frac{|S_v|}{|S|} \) 是子集 \( S_v \) 的权重。

信息增益越大，说明用特征 \( A \) 分裂后数据的不确定性降低得越多。

---

### **4. 选择最优特征**
- 遍历所有特征，计算每个特征的信息增益。
- 选择信息增益最大的特征作为当前节点的分裂特征。

---

### **5. 递归构建子树**
- 对选定的特征 \( A \) 的每个取值 \( v \)：
  - 创建分支，对应 \( A = v \) 的子数据集 \( S_v \)。
  - 如果 \( S_v \) 中所有样

💬 请输入你的消息： quit


✅ 对话已结束
